In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import matplotlib as plt
import glob
import os

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from scipy.signal import medfilt

from skimage.filters import rank
from skimage.morphology import disk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def read_files():
    paths = os.path.join('data', 'processed', 'train', '*')
    return paths

def load_img(path, window_size, fun_name):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = img.astype('float32')
    img /= 255.
    file_name = os.path.basename(file)
    filtered = getattr(fun_name)(img, size=window_size)
    return img, filtered

# Despeckling utils

In [4]:
def lee_filter(img, size):
    img = img.astype('float32')
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance**2 / (img_variance**2 + overall_variance**2)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

def mean_filter(img):
    selem = disk(20)
    percentile_result = rank.mean_percentile(img, selem=selem, p0=.1, p1=.9)
    return percentile_result